## Configuration
_Initial steps to get the notebook ready to play nice with our repository. Do not delete this section._

Code formatting with [black](https://pypi.org/project/nb-black/).

In [1]:
%load_ext lab_black

In [2]:
import os
import pathlib

In [3]:
this_dir = pathlib.Path(os.path.abspath(""))

In [4]:
data_dir = this_dir / "data"

In [5]:
import pytz
import glob
import requests
import pandas as pd
import json
from datetime import datetime

## Download

Retrieve the page

In [6]:
url = "https://services.arcgis.com/rQj5FcfuWPllzwY8/arcgis/rest/services/Mono_Regional_Case_Results/FeatureServer/0//query?where=1%3D1&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&resultType=none&distance=0.0&units=esriSRUnit_Meter&returnGeodetic=false&outFields=Name%2CConfirmed%2CDeaths&returnGeometry=true&returnCentroid=false&featureEncoding=esriDefault&multipatchOption=xyFootprint&maxAllowableOffset=&geometryPrecision=&outSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnExceededLimitFeatures=true&quantizationParameters=&sqlFormat=none&f=pjson&token="

In [7]:
r = requests.get(url)

In [8]:
data = r.json()

## Parse

In [9]:
dict_list = []

In [10]:
for item in data["features"]:
    d = dict(
        county="Mono",
        area=item["attributes"]["Name"],
        confirmed_cases=item["attributes"]["Confirmed"],
    )
    dict_list.append(d)

In [11]:
df = pd.DataFrame(dict_list)

Get timestamp

In [12]:
date_url = "https://services.arcgis.com/rQj5FcfuWPllzwY8/arcgis/rest/services/Mono_Regional_Case_Results/FeatureServer/0?f=json"
date_r = requests.get(date_url)
date_data = date_r.json()

ConnectionError: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))

In [37]:
timestamp = date_data["editingInfo"]["lastEditDate"]

In [38]:
timestamp = datetime.fromtimestamp((timestamp / 1000))

In [39]:
latest_date = pd.to_datetime(timestamp).date()

In [40]:
df["county_date"] = latest_date

In [41]:
exclude = [
    "Mono County",
    "All Regions",
    "Outside Mono County",
    "Marine Warfare Training Center",
]

In [42]:
df = df[~df["area"].isin(exclude)]

Drop Bishop until it reaches 10 cases, then alert

In [43]:
df = df[~(df["confirmed_cases"] <= 10) & ~(df["area"] == "Bishop")]

## Vet

In [44]:
try:
    assert not len(df) > 3
except AssertionError:
    raise AssertionError(
        f"Mono County's scraper has extra {len(export_df) - 9} rows: {list(export_df.area)}"
    )

In [45]:
try:
    assert not len(df) < 3
except AssertionError:
    raise AssertionError(
        f"Mono County's scraper is missing {9 - len(export_df)} rows: {list(export_df.area)}"
    )

## Export

Set date

In [46]:
tz = pytz.timezone("America/Los_Angeles")

In [47]:
today = datetime.now(tz).date()

In [48]:
slug = "mono"

In [49]:
df.to_csv(data_dir / slug / f"{today}.csv", index=False)

## Combine

In [50]:
csv_list = [
    i
    for i in glob.glob(str(data_dir / slug / "*.csv"))
    if not str(i).endswith("timeseries.csv")
]

In [51]:
df_list = []
for csv in csv_list:
    if "manual" in csv:
        df = pd.read_csv(csv, parse_dates=["date"])
    else:
        file_date = csv.split("/")[-1].replace(".csv", "")
        df = pd.read_csv(csv, parse_dates=["county_date"])
        df["date"] = file_date
    df_list.append(df)

In [52]:
df = pd.concat(df_list).sort_values(["date", "area"])

In [53]:
df.to_csv(data_dir / slug / "timeseries.csv", index=False)